In [44]:
import pandas as pd
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import TweetTokenizer
pd.set_option('display.max_colwidth', -1)
from string import punctuation
from nltk.stem.lancaster import LancasterStemmer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import accuracy_score, classification_report,precision_score,recall_score,f1_score,roc_auc_score,confusion_matrix
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
#from imblearn.over_sampling import SMOTE
from nltk.stem import SnowballStemmer

In [3]:
#Load csv
train = pd.read_csv(r'C:\Users\HPPC\train.csv')
test = pd.read_csv(r'C:\Users\HPPC\test.csv')
train["test/train"] = "train"
train=train.dropna()
#print(train.head(10))
test["test/train"] = "test"
train_corpus = train[['tweet','test/train']]
test_corpus = test[['tweet','test/train']]
corpus = pd.concat([train_corpus,test_corpus],ignore_index=True)


In [6]:
stuff_to_be_removed = list(stopwords.words("english"))

In [7]:
#stuff_to_be_removed = list(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()
lancaster_stemmer = LancasterStemmer()
sball_stemmer = SnowballStemmer("english")
cleaned_tweet=[]
def preprocess_word(word):
    # Remove punctuation
    word = word.strip('\'"?!,.():;')
    # Convert more than 2 letter repetitions to 2 letter
    # funnnnny --> funny
    word = re.sub(r'(.)\1+', r'\1\1', word)
    # Remove - & '
    word = re.sub(r'(-|\')', '', word)
    return word


def is_valid_word(word):
    # Check if word begins with an alphabet
    return (re.search(r'^[a-zA-Z][a-z0-9A-Z\._]*$', word) is not None)

def handle_emojis(tweet):
    # Smile -- :), : ), :-), (:, ( :, (-:, :')
    tweet = re.sub(r'(:\s?\)|:-\)|\(\s?:|\(-:|:\'\))', ' EMO_POS ', tweet)
    # Laugh -- :D, : D, :-D, xD, x-D, XD, X-D
    tweet = re.sub(r'(:\s?D|:-D|x-?D|X-?D)', ' EMO_POS ', tweet)
    # Love -- <3, :*
    tweet = re.sub(r'(<3|:\*)', ' EMO_POS ', tweet)
    # Wink -- ;-), ;), ;-D, ;D, (;,  (-;
    tweet = re.sub(r'(;-?\)|;-?D|\(-?;)', ' EMO_POS ', tweet)
    # Sad -- :-(, : (, :(, ):, )-:
    tweet = re.sub(r'(:\s?\(|:-\(|\)\s?:|\)-:)', ' EMO_NEG ', tweet)
    # Cry -- :,(, :'(, :"(
    tweet = re.sub(r'(:,\(|:\'\(|:"\()', ' EMO_NEG ', tweet)
    return tweet

def tweet_cleanining(tweet):
    clean_tweet=[]
    # Convert to lower case
    tweet = tweet.lower()
    # Replaces URLs with the word URL
    tweet = re.sub(r'((www\.[\S]+)|(https?://[\S]+))', 'URL', tweet)
    #Replace {}
    # Replace @handle with the word USER_MENTION
    tweet = re.sub(r'@[\S]+', 'USER_MENTION', tweet)
    # Replaces #hashtag with hashtag
    tweet = re.sub(r'#(\S+)', r' \1 ', tweet)
    # Remove RT (retweet)
    tweet = re.sub(r'\brt\b', '', tweet)
    # Replace 2+ dots with space
    tweet = re.sub(r'\.{2,}', ' ', tweet)
    # Strip space, " and ' from tweet
    tweet = tweet.strip(' "\'')
    # Replace emojis with either EMO_POS or EMO_NEG
    #tweet = handle_emojis(tweet)
    # Replace multiple spaces with a single space
    tweet = re.sub(r'\s+', ' ', tweet)
    #clean_tweet = "Tweet cleaned"
    #Cleaning for each word
    words = tweet.split()

    for word in words:
        word = preprocess_word(word)
        if is_valid_word(word) and word not in stuff_to_be_removed and len(word)>3:
        #    if use_stemmer:
        #        word = str(porter_stemmer.stem(word))
        #    word = word_tokenize(word)
            word = sball_stemmer.stem(word)
            clean_tweet.append(word)

    return ' '.join(clean_tweet)

#new_train = tweet_cleanining(train)

#test_df = train[:30]
for ind in corpus.index: 
    text = tweet_cleanining(corpus['tweet'][ind])
    cleaned_tweet.append(text)
    

corpus['cleaned_tweet'] = cleaned_tweet   
#print(corpus['tweet'],corpus['cleaned_tweet'])    


In [8]:
#Vectorizer for train and test data
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

<H3> TFIDF Modelling </H3>

In [9]:
tfidf=TfidfVectorizer()
vector = tfidf.fit_transform(corpus["cleaned_tweet"])
corpus_X = pd.DataFrame(vector.toarray(),columns=list(tfidf.get_feature_names()))
X_tfidf = corpus_X[0:7273]
X_tfidf_submission = corpus_X[7273:]
y_tfidf = train['sentiment']

In [10]:
print(X_tfidf.shape)
print(X_tfidf_submission.shape)

(7273, 6490)
(1819, 6490)


In [67]:
#Spliting into train and test
X_tfidf_train, X_tfidf_test, y_tfidf_train, y_tfidf_test = tts(X_tfidf,y_tfidf,random_state=42,test_size=0.20,stratify =y_tfidf )

In [12]:
#Applying SMOTE for class imbalance
sm=SMOTE(random_state=0)
X_sm_tfidf_train,y_sm_tfidf_train=sm.fit_sample(X_tfidf_train,y_tfidf_train)

NameError: name 'SMOTE' is not defined

<H2> Logistic Regression </H2>

<h4> Logistic regression without hyperparameters tuning and without SMOTE with class_weight=balanced</h4>

In [47]:
log_reg = LogisticRegression(random_state=42,class_weight="balanced")

In [48]:
log_reg.fit(X_tfidf_train,y_tfidf_train)
print(log_reg.score(X_tfidf_test,y_tfidf_test))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.6597938144329897


In [49]:
y_pred_log = log_reg.predict(X_tfidf_test)

In [50]:
print(classification_report(y_tfidf_test,y_pred_log))

              precision    recall  f1-score   support

           0       0.38      0.32      0.35        88
           1       0.70      0.83      0.76       849
           2       0.64      0.46      0.54       495
           3       0.04      0.04      0.04        23

   micro avg       0.66      0.66      0.66      1455
   macro avg       0.44      0.41      0.42      1455
weighted avg       0.65      0.66      0.65      1455



In [51]:
f1_score(y_tfidf_test,y_pred_log,average='weighted')

0.6483048747515511

In [52]:
precision_score(y_tfidf_test,y_pred_log,average='weighted')

0.6530447830562311

In [53]:
recall_score(y_tfidf_test,y_pred_log,average='weighted')

0.6597938144329897

<h4> Logistic regression without hyperparameters and with SMOTE with class_weight=balanced</h4>

In [129]:
log_reg.fit(X_sm_tfidf_train,y_sm_tfidf_train)
print(log_reg.score(X_tfidf_test,y_tfidf_test))

/Users/deepika/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/deepika/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.6219931271477663


In [130]:
print(classification_report(y_tfidf_test,y_pred_log))

              precision    recall  f1-score   support

           0       0.38      0.32      0.35        88
           1       0.70      0.83      0.76       849
           2       0.64      0.46      0.54       495
           3       0.04      0.04      0.04        23

   micro avg       0.66      0.66      0.66      1455
   macro avg       0.44      0.41      0.42      1455
weighted avg       0.65      0.66      0.65      1455



<h3> Random forest without SMOTE </h3>

In [54]:
rfc=RandomForestClassifier(random_state=42)

In [55]:
rfc.fit(X_tfidf_train,y_tfidf_train)
print(rfc.score(X_tfidf_test,y_tfidf_test))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.6288659793814433


In [56]:
y_rfc_predict = rfc.predict(X_tfidf_test)

In [57]:
print(classification_report(y_tfidf_test,y_rfc_predict))

              precision    recall  f1-score   support

           0       0.47      0.16      0.24        88
           1       0.64      0.87      0.74       849
           2       0.61      0.33      0.43       495
           3       0.00      0.00      0.00        23

   micro avg       0.63      0.63      0.63      1455
   macro avg       0.43      0.34      0.35      1455
weighted avg       0.61      0.63      0.59      1455



In [58]:
precision_score(y_tfidf_test,y_rfc_predict,average='weighted')

0.6113019043528065

In [59]:
recall_score(y_tfidf_test,y_rfc_predict,average='weighted')

0.6288659793814433

In [60]:
f1_score(y_tfidf_test,y_rfc_predict,average='weighted')

0.5905726151427171

<h3> Random forest with SMOTE </h3>

In [175]:
rfc.fit(X_sm_tfidf_train,y_sm_tfidf_train)
print(rfc.score(X_tfidf_test,y_tfidf_test))

0.6350515463917525


<h3> Linear SVC without SMOTE </h3>

In [68]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
#X_s_train = scaler.fit_transform(X_tfidf_train)
svc = LinearSVC(random_state = 42)
svc.fit(X_tfidf_train,y_tfidf_train)
print(svc.score(X_tfidf_test,y_tfidf_test))

0.688659793814433


In [62]:
y_svc_predict = svc.predict(X_tfidf_test)

In [63]:
print(classification_report(y_tfidf_test,y_svc_predict))

              precision    recall  f1-score   support

           0       0.48      0.23      0.31        88
           1       0.70      0.82      0.76       849
           2       0.63      0.52      0.57       495
           3       0.10      0.04      0.06        23

   micro avg       0.67      0.67      0.67      1455
   macro avg       0.48      0.40      0.42      1455
weighted avg       0.65      0.67      0.66      1455



In [64]:
precision_score(y_tfidf_test,y_svc_predict,average='weighted')

0.653819679971677

In [65]:
recall_score(y_tfidf_test,y_svc_predict,average='weighted')

0.6707903780068728

In [66]:
f1_score(y_tfidf_test,y_svc_predict,average='weighted')

0.6551823693435626

In [45]:
confusion_matrix(y_tfidf_test,y_svc_predict)

array([[ 28,  46,  17,   0],
       [  6, 722, 131,   3],
       [  5, 218, 252,   2],
       [  2,  18,   5,   0]], dtype=int64)

<h3> Linear SVC with SMOTE </h3>

In [196]:
svc.fit(X_sm_tfidf_train,y_sm_tfidf_train)
print(svc.score(X_tfidf_test,y_tfidf_test))

0.6164948453608248


In [162]:
from sklearn.pipeline import Pipeline
import numpy as np
SVCpipe = Pipeline([('SVC',LinearSVC())])
param_grid = {'SVC__C':np.arange(0.01,100,10)}

In [198]:
from sklearn.model_selection import GridSearchCV
linearSVC = GridSearchCV(SVCpipe,param_grid,cv=5,return_train_score=True)
linearSVC.fit(X_tfidf_train,y_tfidf_train)
print(linearSVC.best_params_)
#linearSVC.coef_
#linearSVC.intercept_

bestlinearSVC = linearSVC.best_estimator_
bestlinearSVC.fit(X_tfidf_train,y_tfidf_train)
bestlinearSVC.coef_ = bestlinearSVC.named_steps['SVC'].coef_
bestlinearSVC.score(X_tfidf_test,y_tfidf_test)

/Users/deepika/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/deepika/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/deepika/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/deepika/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/deepika/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the num

{'SVC__C': 10.01}


0.643298969072165

In [69]:
y_sub_pred = svc.predict(X_tfidf_submission)

In [70]:
y_sub_pred

array([1, 1, 1, ..., 1, 1, 1], dtype=int64)

In [73]:
predict = pd.DataFrame(y_sub_pred)
predict.to_csv(r'C:\Users\HPPC\Linear SVC Sentiment.csv')